In [ ]:
#import dva_NCP_all_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting
from importlib import reload
from scipy import interpolate
from scipy.optimize import curve_fit


#NCP_days = ['Feb11','Feb12','Feb13']
NCP_days = ['Sep24_p1','Sep24_p2','Sep24_p3','Sep24_p4',
            'Sep25_p1','Sep25_p2','Sep25_p3','Sep25_p4',
            'Sep26_p1','Sep26_p2']
#NCP_days = ['Sep24_p1','Sep24_p2','Sep24_p3','Sep24_p4']
avg_freq = False

## Read in and combine data from list of NCP observations

In [ ]:
%%time

t_set = []
az_set = []
#dec_set = []
#ra_set = []
el_set = []
noise_set = []
#trim_flag = []


# Use one of the scans to get the list of frequencies:
file = h5py.File('/media/ordoga/15m_band1_survey/NCP/'+NCP_days[0]+'_NCP.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

#print(freq)

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

for day in NCP_days:
    print(day)
    # select the file:
    file = h5py.File('/media/ordoga/15m_band1_survey/NCP/'+day+'_NCP.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    #dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    #ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']])
    #trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,:]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,:]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,:]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,::]],axis=0)
    
t_set_plt = Time(t_set, format='isot',scale='utc').mjd

print(RR_set.shape)
print(freq.shape)

## Function to further average over frequency channels (optional)

In [ ]:
def freq_avg(step,i0):

    RR_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    LL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    reRL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    imRL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    freq_avg = np.empty([int(len(freq[i0:-1])/step)])

    for i in range(0,int(len(freq[i0:-1])/step)):
    
        i1 = i0+i*step
        i2 = i0+(i+1)*step
    
        freq_avg[i] = np.nanmean(freq[i1:i2])
        print(freq[i1:i2],freq_avg[i])
    
        RR_avg[:,i] = np.nanmean(RR_set[:,i1:i2],axis=1)
        LL_avg[:,i] = np.nanmean(LL_set[:,i1:i2],axis=1)
        reRL_avg[:,i] = np.nanmean(reRL_set[:,i1:i2],axis=1)
        imRL_avg[:,i] = np.nanmean(imRL_set[:,i1:i2],axis=1)
    
    return RR_avg, LL_avg, reRL_avg, imRL_avg, freq_avg

if avg_freq:
    step = 5
    i0 = 3
    RR_avg, LL_avg, reRL_avg, imRL_avg, freq_avg = freq_avg(step,i0)

## Read in noise cal state and add 2 pixel buffer to exclude leaked timestamps

In [ ]:
wn = list(np.where(noise_set == 1)[0])

for i in range(0,len(wn)):
    wn.append(wn[i]-1)
    wn.append(wn[i]+1)
    wn.append(wn[i]-2)
    wn.append(wn[i]+2)
#print(wn)

if avg_freq:
    reRL_use = reRL_avg.copy()    
    imRL_use = imRL_avg.copy()
    freq_use = freq_avg.copy()
else:
    reRL_use = reRL_set.copy()    
    imRL_use = imRL_set.copy() 
    freq_use = freq.copy()
    
reRL_use[wn,:] = np.nan
imRL_use[wn,:] = np.nan

In [ ]:
dates = ['2022-09-24','2022-09-25','2022-09-26']
#dates = ['2022-09-24']
tsrise = 'T13:50:00.0Z'
tsset = 'T01:50:00.0Z'

srise_arr = []
sset_arr = []

for i in range(0,len(dates)):
    tsrise_plt = Time(dates[i]+tsrise,format='isot',scale='utc').mjd
    tsset_plt = Time(dates[i]+tsset,format='isot',scale='utc').mjd
    srise_arr.append(tsrise_plt)
    sset_arr.append(tsset_plt)
    
    if i ==0:
        wnight = np.where((t_set_plt>tsset_plt) & (t_set_plt<tsrise_plt))[0]
        print(wnight)
    else:
        wnight = np.concatenate([wnight,np.where((t_set_plt>tsset_plt) & (t_set_plt<tsrise_plt))[0]])
        print(wnight)

In [ ]:
print(srise_arr)
print(sset_arr)

## Calculate and subtract median over time for each frequency

In [ ]:
reRL_med_sub = reRL_use.copy()
imRL_med_sub = imRL_use.copy()
reRL_med = []
imRL_med = []
for i in range(0,reRL_use.shape[1]):
    print(freq_use[i])
    reRL_med.append(np.nanmedian(reRL_use[wnight,i]))
    imRL_med.append(np.nanmedian(imRL_use[wnight,i]))
    reRL_med_temp = reRL_use[:,i] - reRL_med[i]
    imRL_med_temp = imRL_use[:,i] - imRL_med[i]
    #print(min(reRL_med_temp),max(reRL_med_temp))
    
    w1 = np.where(np.isfinite(reRL_med_temp))[0]
    w2 = np.where(np.isfinite(imRL_med_temp))[0]

    if len(w1) > 0:
        f1 = interpolate.interp1d(t_set_plt[w1], reRL_med_temp[w1])
        reRL_med_sub[:,i] = f1(t_set_plt)
    else:
        reRL_med_sub[:,i] = reRL_med_temp
    if len(w2) > 0:
        f2 = interpolate.interp1d(t_set_plt[w2], imRL_med_temp[w2])
        imRL_med_sub[:,i] = f2(t_set_plt)  
    else:
        imRL_med_sub[:,i] = imRL_med_temp
    #print(min(reRL_med_sub[:,i]),max(reRL_med_sub[:,i]))
    #print(i)

## Plot median-subtracted reRL and imRL vs time for selected frequency

In [ ]:
fig1, axs = plt.subplots(3,1,figsize=(20,14),sharex=True)
fs = 12

fpick = 930
df = freq_use[1]-freq_use[0]
wf = np.where(abs(freq_use-fpick)<df/2.)[0]
print(wf, freq_use[wf])


def sine_func(t, A, B, t0, D):
    return A * np.sin(B*(t-t0)) + D

#wuse = np.intersect1d(wkeep,wnight)
wuse = wnight.copy()

#print(len(wkeep))
print(len(wnight))
print(len(wuse))

# re(RL) pol. fits:

#p0 = [0.01,  2.*np.pi/0.5, t_set_plt[0], 0]
#wfin1 = np.where(np.isfinite(reRL_med_sub[wnight,wf]))
#popt_reRL,pcov = curve_fit(sine_func,t_set_plt[wuse][wfin1],reRL_med_sub[wnight,wf][wfin1],p0=p0)
#print('corrected reRL')
#print('Amplitude: ',popt_reRL[0])
#print('Period: ',24.*2.*np.pi/popt_reRL[1])
#print('Phase: ',popt_reRL[2])
#print('Offset: ',popt_reRL[3])
#print('')

# im(RL) pol. fits:

#p0 = [0.01,  2.*np.pi/0.5, t_set_plt[0], 0]
#wfin1 = np.where(np.isfinite(imRL_med_sub[wnight,wf]))
#popt_imRL,pcov = curve_fit(sine_func,t_set_plt[wuse][wfin1],imRL_med_sub[wnight,wf][wfin1],p0=p0)
#print('corrected imRL')
#print('Amplitude: ',popt_imRL[0])
#print('Period: ',24.*2.*np.pi/popt_imRL[1])
#print('Phase: ',popt_imRL[2])
#print('Offset: ',popt_imRL[3])
#print('')



axs[0].scatter(t_set_plt,reRL_med_sub[:,wf],s=1,color='C0')
axs[0].scatter(t_set_plt[wnight],reRL_med_sub[wnight,wf],s=1,color='blue')
axs[1].scatter(t_set_plt,imRL_med_sub[:,wf],s=1,color='C1')
axs[1].scatter(t_set_plt[wnight],imRL_med_sub[wnight,wf],s=1,color='red')
axs[2].scatter(t_set_plt,(180./np.pi)*0.5*np.arctan2(imRL_med_sub[:,wf],reRL_med_sub[:,wf]),s=0.01,color='grey')
axs[2].scatter(t_set_plt[wnight],(180./np.pi)*0.5*np.arctan2(imRL_med_sub[wnight,wf],reRL_med_sub[wnight,wf]),
               s=0.01,color='green')

#axs[0].plot_date(t_set_plt, sine_func(t_set_plt,*popt_reRL),color='black',xdate=True,fmt='-')
#axs[1].plot_date(t_set_plt, sine_func(t_set_plt,*popt_imRL),color='black',xdate=True,fmt='-')

axs[0].set_title('1020 MHz',fontsize = 20)

for i in range(0,3):
    #axs[i].set_ylim(-0.5,0.5)
    if i < 2:
        #pass
        axs[i].set_ylim(-5e4,5e4)
    else:
        axs[i].set_ylim(-90,90)
    axs[i].set_xlim(t_set_plt[0],t_set_plt[-1])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)
    #axs[1].set_ylim(-0.03,0.03)
    
    
#plt.savefig('../DVA2/PLOTS/NCP_408MHz.jpg')

## Plot the medians of reRL and imRL vs freq (instrumental polarization)

In [ ]:
fig1, axs = plt.subplots(4,1,figsize=(20,16),sharex=True)

fs = 12

ii = 5000

axs[0].scatter(freq_use,reRL_med,s=20)
axs[0].scatter(freq_use,reRL_use[ii,:],s=20)
axs[0].scatter(freq_use,reRL_med_sub[ii,:],s=20)
axs[1].scatter(freq_use,imRL_med,s=20)
axs[1].scatter(freq_use,imRL_use[ii,:],s=20)
axs[2].scatter(freq_use,(180/np.pi)*0.5*np.arctan2(imRL_med,reRL_med),s=20)
axs[3].scatter(freq_use,np.sqrt(np.asarray(imRL_med)**2 + np.asarray(reRL_med)**2),s=20)

axs[0].set_ylim(-0.5,0.5)
axs[1].set_ylim(-0.5,0.5)
axs[2].set_ylim(-90,90)
axs[3].set_ylim(0,1)

for i in range(0,4):
    axs[i].set_xlim(350,1030)
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].grid()

In [ ]:
%%time

#Gmin = 0.9
#Gmax = 1.1
noisemin = 60
noisemax = 80
dBmin = 60
dBmax = 80

fplot = 800
df = freq[1]-freq[0]
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])

i1 = 0
i2 = -1

fig1, axs = plt.subplots(3,1,figsize=(20,25),sharex=True,gridspec_kw={'height_ratios': [1,1,1]},
                         constrained_layout=True)
fs = 16

extent = [t_set_plt[i1],t_set_plt[i2],freq[0],freq[-1]]
    
im0 = axs[0].imshow(reRL_med_sub[i1:i2,:].T,aspect='auto',vmin=-0.02,vmax=0.02,origin='lower',
                    extent=extent,cmap='bwr')    
im1 = axs[1].imshow(imRL_med_sub[i1:i2,:].T,aspect='auto',vmin=-0.02,vmax=0.02,origin='lower',
                    extent=extent,cmap='bwr')
im2 = axs[2].imshow((180./np.pi)*0.5*np.arctan2(imRL_med_sub[i1:i2,:],reRL_med_sub[i1:i2,:]).T,aspect='auto',
                    vmin=-90,vmax=90,origin='lower',extent=extent,cmap='twilight')


cbar0= fig1.colorbar(im0,ax=axs[0])
cbar0.ax.tick_params(labelsize=fs) 
cbar0.set_label('', fontsize=fs)

cbar1= fig1.colorbar(im1,ax=axs[1])
cbar1.ax.tick_params(labelsize=fs) 
cbar1.set_label('', fontsize=fs)

cbar2= fig1.colorbar(im2,ax=axs[2])
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('', fontsize=fs)

for i in range(0,3):
    axs[i].axvline(x = srise_arr[i])
    axs[i].axvline(x = sset_arr[i])
    axs[i].set_xlim(t_set_plt[i1],t_set_plt[i2])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)
    axs[i].set_ylabel('Frequency (MHz)',fontsize=fs)
    

In [ ]:
ii = 6000
print(t_set[ii])

plt.plot(freq,(180./np.pi)*0.5*np.arctan2(imRL_med_sub[ii,:],reRL_med_sub[ii,:]))

In [ ]:
%%time

i1 = 0
i2 = -1

fig1, axs = plt.subplots(1,1,figsize=(20,22))
fs = 16

extent = [t_set_plt[i1],t_set_plt[i2],freq[0],freq[-1]]

im2 = axs.imshow(reRL_med_sub[i1:i2,:].T,aspect='auto',vmin=-0.02,vmax=0.02,origin='lower',
                    extent=extent,cmap='bwr') 

cbar2= fig1.colorbar(im2,ax=axs)
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('', fontsize=fs)

axs.set_xlim(t_set_plt[i1],t_set_plt[i2])
axs.tick_params(axis='both', labelsize=fs,labelbottom=True)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.set_xlabel('Time (UTC)',fontsize=fs)
axs.set_ylabel('Frequency (MHz)',fontsize=fs)

for i in range(0,len(sset_arr)):
    axs.axvline(x = srise_arr[i], color='black', linewidth=1)
    axs.axvline(x = sset_arr[i], color='black', linewidth=1)

plt.savefig('../DVA2/PLOTS/NCP_Q_waterfall.jpg')

In [ ]:
%%time

i1 = 0
i2 = -1

fig1, axs = plt.subplots(1,1,figsize=(20,22))
fs = 16

extent = [t_set_plt[i1],t_set_plt[i2],freq[0],freq[-1]]

im2 = axs.imshow(imRL_med_sub[i1:i2,:].T,aspect='auto',vmin=-0.02,vmax=0.02,origin='lower',
                    extent=extent,cmap='bwr') 

cbar2= fig1.colorbar(im2,ax=axs)
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('', fontsize=fs)

axs.set_xlim(t_set_plt[i1],t_set_plt[i2])
axs.tick_params(axis='both', labelsize=fs,labelbottom=True)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.set_xlabel('Time (UTC)',fontsize=fs)
axs.set_ylabel('Frequency (MHz)',fontsize=fs)

for i in range(0,len(sset_arr)):
    axs.axvline(x = srise_arr[i], color='black', linewidth=1)
    axs.axvline(x = sset_arr[i], color='black', linewidth=1)

plt.savefig('../DVA2/PLOTS/NCP_U_waterfall.jpg')

In [ ]:
%%time

i1 = 0
i2 = -1

fig1, axs = plt.subplots(1,1,figsize=(20,22))
fs = 16

extent = [t_set_plt[i1],t_set_plt[i2],freq[0],freq[-1]]

im2 = axs.imshow((180./np.pi)*0.5*np.arctan2(imRL_med_sub[i1:i2,:],reRL_med_sub[i1:i2,:]).T,aspect='auto',
                    vmin=-90,vmax=90,origin='lower',extent=extent,cmap='twilight')

cbar2= fig1.colorbar(im2,ax=axs)
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('', fontsize=fs)

axs.set_xlim(t_set_plt[i1],t_set_plt[i2])
axs.tick_params(axis='both', labelsize=fs,labelbottom=True)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.set_xlabel('Time (UTC)',fontsize=fs)
axs.set_ylabel('Frequency (MHz)',fontsize=fs)

for i in range(0,len(sset_arr)):
    axs.axvline(x = srise_arr[i], color='yellow', linewidth=1)
    axs.axvline(x = sset_arr[i], color='yellow', linewidth=1)

plt.savefig('../DVA2/PLOTS/NCP_angle_waterfall_uncorrected.jpg')

In [ ]:
lbd2 = ((3e8)/(freq_avg*1e6))**2
print(lbd2)

In [ ]:
ii = 31000

plt.scatter(lbd2,0.5*np.arctan2(imRL_avg_fix_med[ii,:],reRL_avg_fix_med[ii,:]))
plt.xlim(0.1,0.2)

In [ ]:
print(2.5/(0.19-0.1))

In [ ]:
for i in range(0,len(freq_avg)):
    print(freq_avg[i],lbd2[i])

## Testing .h5 file formats for earlier stages of data collection and spectrometer config.

In [ ]:
file = h5py.File('/home2/DATA_AO/DVA_DATA/Mar09_NCP/2022-03-09T00:00:30.599989Z.h5','r')

band_ids = file['data']['beam_0'].keys()
for band_id in band_ids:
    print(band_id)
dataset = file['data']['beam_0']['band_SB7']
print(dataset.keys())
print('')

freq = dataset['frequency']
print('frequency')
print(freq[:])
print(freq[1]-freq[0])
print('')

data = dataset['scan_0']
print(data.keys())
print('')

position = data['position']
print('el            az          latitude       longitude')
print(position[0])
print(position[1])
print(position[2])
print('')

time = data['time']
print('time')
print(time[0])
print(time[1])
print(time[2])
print('')

power = data['data']
print('data array')
print(power.shape)

#print(dataset['metadata'])
#print(dataset.keys())
#print(dataset['position'][0])
#print(file['data']['beam_0']['band_SB3']['frequency'][:])

In [ ]:
file = h5py.File('/home2/DATA_AO/DVA_DATA/Feb11_NCP_part1/2022-02-11T00:00:54.659988Z.h5','r')

dataset = file['data']['beam_0']['band_SB7']
print(dataset.keys())
print('')

freq = dataset['frequency']
print('frequency')
print(freq[:])
print((freq[1]-freq[0])/1e6)
print('')


metadata = dataset['scan_0']['metadata']
print(metadata)
print('ra',metadata['right_ascension'][0:3])
print('dec',metadata['declination'][0:3])
print('az',metadata['azimuth'][0:3])
print('el',metadata['elevation'][0:3])
print('utc',metadata['utc'][0:3])
#print('noise',metadata['noise_state'])
print('')

data = dataset['scan_0']['data']
print('data array')
print(data.shape)
print('')
